# Writing an Algorithm that Uses DMAC Trading

## Data Loading

In [ ]:
# Initial imports
import numpy as np
import os
import csv
import requests
import pandas as pd
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
from MCForecastTools import MCSimulation
import numpy as np
import matplotlib.pyplot as plt
import seaborn
import warnings
from pathlib import Path
import datetime as dt

#Subsequent imports
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf
import hvplot.pandas

# Enable the Matplotlib property to allow diagrams to display in Jupyter Notebook
%matplotlib inline

In [ ]:
pip install alpaca-trade-api

In [ ]:
pip install yfinance

In [ ]:
# List the ticker symbols for the stocks of interest
lst=['BMO.TO', 'BNS.TO', 'CM.TO', 'NA.TO','RY.TO', 'TD.TO', 'ZEB.TO','ZWB.TO', 'XGB.TO', 'CL' ]

# Determine the start and end dates
start_date = '2022-01-01'
end_date = '2022-11-30'

# Create empty dataframe to hold stock data
dfAllStocks_Yahoo=pd.DataFrame()

# Gathers stock data from listed tickers and combines them into a dataframe
for ticker in lst:
    # Create data frame from API in loop 
    df= yf.download(ticker, start= start_date, end= end_date)
    # Remove to '.TO' suffix from Canadian stocks
    tic= ticker.replace('.TO','')
    df['Ticker']=tic
    # Drop unwanted coloumns - Keep Adjusted Close
    df.drop(columns= ["Open", "High", "Low", "Volume", "Close", "Ticker"], inplace= True)
    # Combine individual stocks into a single data frame
    dfAllStocks_Yahoo=pd.concat([dfAllStocks_Yahoo, df], axis= 1)
    
  # Rename the coloumns to the ticker symbols in the list
dfAllStocks_Yahoo.columns = lst
dfAllStocks_Yahoo.index.name = 'Date'

# Drop the N/As
dfAllStocks_Yahoo = dfAllStocks_Yahoo.dropna()

# Drop the time component of the date
dfAllStocks_Yahoo.index = dfAllStocks_Yahoo.index.date

# Drop the time component of the date
#dfAllStocks_Yahoo.index = dfAllStocks_Yahoo.index.date

# Display the data
display(dfAllStocks_Yahoo)

In [ ]:
# Filter the date index and close columns
signals_df = dfAllStocks_Yahoo.loc[:,["BNS.TO"]]

# Review the DataFrame
signals_df.head()

In [ ]:
# Use hvplot to visualize the data
signals_df.hvplot()

## SMA for the Short and Long Windows Calculation

In [ ]:
 # Set the variables for short window and long window periods via the for loop below to determine the maximum cumulative return amount 
    
dctStats={'ShortMA':[],'LongMA':[],'CumRtn':[]}

for long in range(1,100,1):   
    for short in range(1,100,1):
        short_window = short
        long_window = long
        
        print('------------------ The short MA is '+str(short_window) +', the long window is '+str(long))
        
        # Generate the short and long window simple moving averages (starting at 1 thru 100, at 1 day increments for each curve at a time)
        signals_df["SMAShort"] = signals_df["BNS.TO"].rolling(window=short_window).mean()
        signals_df["SMALong"] = signals_df["BNS.TO"].rolling(window=long_window).mean()

        # Review the DataFrame
        #display(signals_df.head())
        #display(signals_df.tail())

        # Create a column to hold the trading signal
        signals_df["Signal"] = 0.0

        # Generate the trading signal 0 or 1,
        # where 1 is the short-window (SMA50) greater than the long-window (SMA100)
        # and 0 is when the condition is not met
        signals_df["Signal"][short_window:] = np.where(
            signals_df["SMAShort"][short_window:] > signals_df["SMALong"][short_window:], 1.0, 0.0)
        
        # Set initial capital
        initial_capital = float(100000)

        # Set the share size
        share_size = 500

        # Buy a 500 share position when the dual moving average crossover Signal equals 1
        # Otherwise, `Position` should be zero (sell)
        signals_df['Position'] = share_size * signals_df['Signal']
        
        #display(signals_df)

        # Determine the points in time where a 500 share position is bought or sold
        signals_df['Entry/Exit Position'] = signals_df['Position'].diff()

        # Multiply the close price by the number of shares held, or the Position
        signals_df['Portfolio Holdings'] = signals_df['BNS.TO'] * signals_df['Position']

        # Subtract the amount of either the cost or proceeds of the trade from the initial capital invested
        signals_df['Portfolio Cash'] = initial_capital - (signals_df['BNS.TO'] * signals_df['Entry/Exit Position']).cumsum()

        # Calculate the total portfolio value by adding the portfolio cash to the portfolio holdings (or investments)
        signals_df['Portfolio Total'] = signals_df['Portfolio Cash'] + signals_df['Portfolio Holdings']

        # Calculate the portfolio daily returns
        signals_df['Portfolio Daily Returns'] = signals_df['Portfolio Total'].pct_change()

        # Calculate the portfolio cumulative returns
        signals_df['Portfolio Cumulative Returns'] = (1 + signals_df['Portfolio Daily Returns']).cumprod() - 1
        
        dctStats['ShortMA'].append(short_window)
        dctStats['LongMA'].append(long_window)
        dctStats['CumRtn'].append(signals_df['Portfolio Cumulative Returns'][-1])
        
        #display(signals_df)
        print(signals_df['Portfolio Cumulative Returns'][-1])
                
dfStats=pd.DataFrame(dctStats)

dfStats=dfStats.sort_values(['CumRtn'], ascending=False)

display(dfStats)
        

## Cumulative Returns

In [ ]:
# Calculate cumulative returns
portfolio_evaluation_df.loc["Cumulative Returns"] = signals_df["Portfolio Cumulative Returns"][-1]

# Review the result
portfolio_evaluation_df

In [ ]:
## Annual Volatility

In [ ]:
# Calculate annual volatility
portfolio_evaluation_df.loc["Annual Volatility"] = (
    signals_df["Portfolio Daily Returns"].std() * np.sqrt(252)
)

# Review the result
portfolio_evaluation_df
                            

In [ ]:
## Sharpe Ratio

In [ ]:
# Calculate Sharpe Ratio
portfolio_evaluation_df.loc["Sharpe Ratio"] = (
    signals_df["Portfolio Daily Returns"].mean() * 252) / (
    signals_df["Portfolio Daily Returns"].std() * np.sqrt(252)
)

# Review the result
portfolio_evaluation_df

In [ ]:
## Sortino Ratio

In [ ]:
# Calculate downside return values

In [ ]:
# Create a DataFrame that contains the Portfolio Daily Returns column
sortino_ratio_df = signals_df[["Portfolio Daily Returns"]].copy()

# Create a column to hold downside return values
sortino_ratio_df.loc[:,"Downside Returns"] = 0

# Find Portfolio Daily Returns values less than 0,
# square those values, and add them to the Downside Returns column
sortino_ratio_df.loc[sortino_ratio_df["Portfolio Daily Returns"] < 0,
                     "Downside Returns"] = sortino_ratio_df["Portfolio Daily Returns"]**2

# Review the DataFrame
sortino_ratio_df.tail()

In [ ]:
# Calculate the Sortino Ratio

In [ ]:
# Calculate the annualized return value
annualized_return = (
    sortino_ratio_df["Portfolio Daily Returns"].mean() * 252
)

# Print the result
print(f"Annualized Return: {annualized_return}")

In [ ]:
# Calculate the annualized downside standard deviation value
downside_standard_deviation = (
    np.sqrt(sortino_ratio_df["Downside Returns"].mean()) * np.sqrt(252)
)

# Print the result
print(f"Annualized Downside STD: {downside_standard_deviation}")

In [ ]:
# The Sortino ratio is reached by dividing the annualized return value
# by the downside standard deviation value
sortino_ratio = annualized_return/downside_standard_deviation

# Add the Sortino ratio to the evaluation DataFrame
portfolio_evaluation_df.loc["Sortino Ratio"] = sortino_ratio

# Review the DataFrame
portfolio_evaluation_df

In [ ]:
## Set Up the Trade-Level Risk/Reward Evaluation Metrics

In [ ]:
'''# Initialize trade evaluation DataFrame with columns
trade_evaluation_df = pd.DataFrame(
    columns=[
        "Stock",
        "Entry Date",
        "Exit Date",
        "Shares",
        "Entry Share Price",
        "Exit Share Price",
        "Entry Portfolio Holding",
        "Exit Portfolio Holding",
        "Profit/Loss"]
)'''

In [ ]:
'''# Loop through signal DataFrame
# If `Entry/Exit` is 1, set entry trade metrics
# Else if `Entry/Exit` is -1, set exit trade metrics and calculate profit
# Then append the record to the trade evaluation DataFrame
for index, row in signals_df.iterrows():
    #display(signals_df).head(500)
    if row["Entry/Exit"] == 1:
        entry_date = index
        entry_portfolio_holding = row["Portfolio Holdings"]
        #print(entry_portfolio_holding)
        share_size = row["Entry/Exit Position"]
        entry_share_price = row["BNS.TO"]

    elif row["Entry/Exit"] == -1:
        exit_date = index
        exit_portfolio_holding = abs(row["BNS.TO"] * row["Entry/Exit Position"])
        exit_share_price = row["BNS.TO"]
        profit_loss = exit_portfolio_holding - entry_portfolio_holding
        trade_evaluation_df = trade_evaluation_df.append(
            {
                "Stock": "AAPL",
                "Entry Date": entry_date,
                "Exit Date": exit_date,
                "Shares": share_size,
                "Entry Share Price": entry_share_price,
                "Exit Share Price": exit_share_price,
                "Entry Portfolio Holding": entry_portfolio_holding,
                "Exit Portfolio Holding": exit_portfolio_holding,
                "Profit/Loss": profit_loss
            },
            ignore_index=True)

# Print the DataFrame
trade_evaluation_df'''